In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cm import get_cmap
import seaborn as sns
import string
import pickle # save data frame (results) in a .pkl file
import pandas as pd
from datetime import datetime
import os, sys
import re

import grakel as gk


sys.path.insert(0, 'C:/Users/User/Code/MMDGraph')
sys.path.insert(0, 'C:/Users/User/Code/MMDGraph/myKernels')
#from importlib import reload
#reload(readfoldertopanda)
import mmdutils
import importlib
importlib.reload(sys.modules['mmdutils'])
from mmdutils import readfoldertopanda, PlotROCGeneral, plotVaryingBGDEG, findAUC
import MMDforGraphs as mg
importlib.reload(sys.modules['MMDforGraphs'])
from myKernels import RandomWalk as rw
import WL

Generate Weighted graphs with scale free structure

In [2]:
from scipy.stats import uniform
uniform.rvs(size=1,  loc = 0.4 , scale = 0.5)[0]

0.5621775472391118

In [9]:
n1 = 10
nnode1 = 5
k1 = 2.1

n2 = 10
nnode2 = 5
k2 = 2.1


g1 = mg.ScaleFreeGraph(n1, nnode1, k1, fullyConnected = True, e = 'random_edge_weights', ul = 0.4, uu =0.7)
g2 = mg.ScaleFreeGraph(n2, nnode2, k2, fullyConnected = True, e = 'random_edge_weights', ul = 0.4, uu =0.5)


In [10]:
importlib.reload(sys.modules['MMDforGraphs'])
g1.Generate()
g2.Generate()
Gs = g1.Gs + g2.Gs

# Grakel kernels
# graph_list = gk.graph_from_networkx(Gs, node_labels_tag='label')  # Convert to graphs to Grakel format
# kernel = [{"name": "multiscale_laplacian"}]
# K = mg.KernelMatrix(graph_list, kernel, False)   
#kernel = [{"name": "weisfeiler_lehman", "n_iter": 4}, {"name": "vertex_histogram"}]
#kernel = [{"name": "WL-OA", "n_iter": 4}]
# kernel = [{"name": "SP", "with_labels": 1}]
# kernel = [{"name": "lovasz_theta", "n_samples":20}]
#kernel = [{"name": "core_framework", "min_core": 1}, {"name":"propagation", 't_max':5, 'w':0.1, 'M':"H"}]
# kernel = [{"name": "pyramid_match", "with_labels":False, "d":2, "L":4}] 
#kernel = [{"name":'RW', 'kernel_type':'geometric', 'lamda':0.1, 'with_labels':False, 'p':3}]
# kernel = [{"name":'odd_sth', 'h':3}]
#kernel = [{"name": "weisfeiler_lehman", "n_iter": 1}, {"name": "propagation", "t_max": 2, "w":0.01, "M":'TV'}]

#init_kernel = gk.GraphKernel(kernel= kernel, normalize=1, n_jobs= 4)
#K = init_kernel.fit_transform(graph_list)

# My kernels
# RW kernel
# label list for rw
label_list = []
for G in Gs:
    label_list.append(np.unique(list(nx.get_node_attributes(G, 'label').values())))

# label_list = np.unique(np.concatenate(label_list))
rw_kernel = rw.RandomWalk(Gs, c = 0.01, normalize=0 )
# K = rw_kernel.fit_ARKL(r = 8, label_list = label_list, normalize_adj=True)
K = rw_kernel.fit_ARKU_plus(r = 12, normalize_adj=False)
# K = rw_kernel.fit_random_walk(mu_vec=[0.0, 1.0/(nnode_1*nnode_2)], k = 1, r = 3, normalize_adj=False ) # 1/((nnode_1-1)*(nnode_2-1)))

print(np.all(K == 0)) 
print(K)

MMD_functions = [mg.MMD_b, mg.MMD_u]

# initialize bootstrap class, we only want this to be initalized once so that numba njit
# only gets compiled once (at first call)
kernel_hypothesis = mg.BoostrapMethods(MMD_functions)
function_arguments=[dict(n = g1.n, m = g2.n ), dict(n = g1.n, m = g2.n )]
kernel_hypothesis.Bootstrap(K, function_arguments, B = 3000, method = 'BootstrapScheme')

# Run power
#threshold = np.quantile(kernel_hypothesis.boot_test_statistic['MMD_u'], 1-0.05)
#print(f'threshold {threshold}')
#ratio, power, V = mg.power_ratio(K, kernel_hypothesis.sample_test_statistic['MMD_u'], threshold, n_1)
#print(f'V {V}')
#print(f'ratio {ratio}')
#print(f'power {power}')
print(f'p_value {kernel_hypothesis.p_values}')
print(f"MMD_u {kernel_hypothesis.sample_test_statistic['MMD_u']}")

  0%|          | 0/210.0 [00:00<?, ?it/s]C:/Users/User/Code/MMDGraph\myKernels\RandomWalk.py:903: FutureWarning: attr_matrix will return an numpy.ndarray instead of a numpy.matrix in NetworkX 3.0.
  return scipy.sparse.csr_matrix(np.array(nx.attr_matrix(G, edge_attr=edge_attr)[0]), dtype=np.float64)
c:\Users\User\Code\MMDGraph\.mmdvenv8\lib\site-packages\scipy\sparse\linalg\_eigen\arpack\arpack.py:1592: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.

In [8]:
np.max(K)

0.07550579738028522